# 최소신장트리

가중치 그래프에서 모든 정점들을 연결하는 간선들의 가중치의 합이 최소가 되는 트리를 찾는 문제<br><br>
- 신장 트리 : n개의 정점을 포함하는 무향 그래프에서 **n개의 정점과 n-1개의 간선**으로 구성된 트리<br>
- 최소 신장 트리 : 가중치 그래프에서 신장 트리를 구성하는 간선들의 가중치의 합이 최소인 신장 트리

## 프림 알고리즘

한 정점에 연결된 간선들 중 하나씩 선택하면서 최소 신장 트리를 만들어 가는 방식
1. 임의의 정점을 하나 선택해서 시작
2. 선택한 정점들과 인접하는 정점들 중에 최소 비용의 간선이 존재하는 정점을 선택
3. 모든 정점이 선택될 때까지 두번째 과정 반복

### heapq 사용

In [1]:
myedges = [
    (7, 'A', 'B'), (5, 'A', 'D'),
    (8, 'B', 'C'), (9, 'B', 'D'),
    (7, 'B', 'E'), (5, 'C', 'E'),
    (7, 'D', 'E'), (6, 'D', 'F'),
    (8, 'E', 'F'), (9, 'E', 'G'),
    (11, 'F', 'G')
]

In [2]:
from collections import defaultdict
from heapq import *

def prim(start_node, edges):
    mst = [] # 최소신장트리
    adjacent_edges = defaultdict(list) # 모든 간선 정보를 저장할 딕셔너리
    for weight, n1, n2 in edges:
        adjacent_edges[n1].append((weight, n1, n2))
        adjacent_edges[n2].append((weight, n1, n2))
        
    connected_nodes = set(start_node) # 연결된 노드의 집합. start_node는 바로 삽입.
    candidate_edge_list = adjacent_edges[start_node] # 간선리스트에 현재 노드와 연결된 간선들을 삽입.
    heapify(candidate_edge_list) # 최소 가중치를 가지는 간선부터 추출하기 위해, heapq 로 변경.
    
    # 간선 리스트에 더 이상 간선이 없을 때까지
    while candidate_edge_list:
        weight, n1, n2 = heappop(candidate_edge_list) # 최소 가중치를 가지는 노드 pop
        if n2 not in connected_nodes: # 해당 노드에 연결된 인접정점이 연결 노드 집합에 들어있지 않다면(사이클 방지)
            connected_nodes.add(n2) # 연결노드 집합에 해당 인접 정점을 더한다.
            mst.append((weight, n1, n2)) # mst에 해당 간선 정보를 삽입한다.
            
            for edge in adjacent_edges[n2]: # 해당 인접정점의 간선정보를 순회하는데,
                if edge[2] not in connected_nodes: # 인접정점에 연결된 간선 노드가 연결된 노드 집합에 없다면
                    heappush(candidate_edge_list, edge) # 간선리스트에 해당 간선 정보를 삽입
    return mst

In [3]:
prim('A', myedges)

[(5, 'A', 'D'),
 (6, 'D', 'F'),
 (7, 'A', 'B'),
 (7, 'B', 'E'),
 (8, 'B', 'C'),
 (9, 'E', 'G')]

### heapq 없는 풀이

In [33]:
graph = {
    0: [(1, 32),(2, 31),(5, 60),(6, 51)],
    1: [(0, 32),(2, 21)],
    2: [(0, 31),(1, 21),(4, 46),(6, 25)],
    3: [(4, 34),(5, 18)],
    4: [(2, 46),(3, 34),(5, 40),(6, 51)],
    5: [(0, 60),(3, 18),(4, 40)],
    6: [(0, 51),(2, 25),(4, 51)]    
}

In [41]:
# G: 그래프, s: 시작 정점
def mst_prim(G, s):
    N = len(G)
    key = [float('inf')] * N # 가중치를 무한대로 초기화
    pi = [None]*N # 트리에서 연결될 부모 정점 초기화
    visited = [False]*N # 방문 여부 초기화
    key[s] = 0 # 시작 정점의 가중치를 0으로 설정
    
    for _ in range(N): # 정점의 개수만큼 반복
        min_idx = -1
        min_val = float('inf')
        for i in range(N):
            if not visited[i] and key[i] < min_val:
                min_val = key[i]
                min_idx = i
        visited[min_idx] = True # 최소 가중치 정점 방문처리
            
        for v, val in G[min_idx]: # 선택 정점의 인접한 정점에 대해서
            if not visited[v] and val < key[v]:
                key[v] = val # 가중치 갱신
                pi[v] = min_idx # 트리에서 연결될 부모 정점 갱신
    print(key)
    print(pi)

In [42]:
mst_prim(graph, 0)

[0, 21, 31, 34, 46, 18, 25]
[None, 2, 0, 4, 2, 3, 2]


## 크루스칼 알고리즘

### union-find 알고리즘

Disjoint Set(서로소 집합)을 표현할 때 사용하는 알고리즘으로 트리 구조를 활용하는 알고리즘.<br>
즉, 노드들 중에서 연결된 노드를 찾거나, 노드들을 서로 연결할 때(합칠때) 사용<br>


### 해설 1

N개의 정점을 포함하는 그래프에서 n-1개의 간선을 선택하는 방식<br>
프림 알고리즘은 하나의 트리를 확장시켜 가는 방식이지만 크루스칼 알고리즘은 간선을 선택해 나가는 과정에 여러개의 간선이 존재.<br>
간선을 선택해 나가는 과정에 여러개의 트리들이 존재한다.<br><br>

초기 상태는 n개의 정점들이 각각 하나의 트리가 됨 -> 하나의 정점을 포함하는 n개의 상호 배타 집합 존재<br>
간선을 선택하면 간선의 두 정점이 속한 트리가 연결되고, 하나의 집합으로 합쳐짐<br>
선택한 간선의 두 정점이 이미 연결된 트리에 속한 정점들일 경우 사이클이 생김 -> 두 정점에 대해 같은 집합의 원소인지 여부를 검사한다.
<br><br>

1. 최초, 모든 간선을 가중치에 따라 **오름차순** 정렬
2. 가중치가 **가장 낮은 간선부터 선택**하면서 트리를 증가시킴 (사이클이 존재하면 다음으로 가중치가 낮은 간선 선택)
3. n-1개의 간선이 선택될 때까지 두번째 과정을 반복

In [3]:
# 수도코드
def mst_kruskal(g):
    mst = [] # 최소신장트리를 구성하는 간선들의 집합
    
    for i in range(N): # 하나의 정점만 포함하는 상호배타 집합을 정점의 수만큼 생성
        Make_Set(i)
    
    g.sort(key=lambda x:x[2]) # 모든 간선들을 가중치 순으로 정렬
    
    mst_cost = 0
    
    while len(mst) < N-1:
        u, v, val = G.pop(0) # 간선의 가중치가 가장 낮은 간선을 선택
        if Find_Set(u) != Find_Set(v): # 정점 u와 v가 서로 다른 집합의 원소인지 확인. 서로 같은 집합에 있으면 사이클 생김
            Union(u, v) # 정점 u와 v가 속한 집합을 합침
            mst.append((u, v)) # 선택한 간선을 집합 mst에 추가
            mst_cost += val

- 간선 선택 과정에서 생성되는 트리를 관리하기 위해 **상호 배타 집합 사용**
    - 트리에 속한 노드들은 자신을 루트로 하는 서브트리의 높이를 rank 라는 이름으로 저장
- 선택한 간선으로 두 개의 트리가 한 개의 트리로 합쳐질 때 각 트리에 해당하는 상호 배타 집합을 **Union 연산**으로 합침
    - 랭크 값이 작은 트리를 랭크 값이 큰 트리의 서브트리로 포함시킬 경우 트리에 포함된 노드들의 랭크 값 수정 불필요

In [ ]:
# input
"""
6 11
0 1 32
0 2 31
0 5 60
0 6 51
1 2 21
2 4 46
2 6 25
3 4 34
3 5 18
4 5 40
4 6 51
"""

In [8]:
def find_set(x): # 부모를 찾아가는 과정
    while x != p[x]:
        x = p[x] # x가 가리키는 정점으로 이동
    return x

V, E = map(int, input().split()) # 노드 개수, 간선 개수
edge = []
for _ in range(E):
    u, v, w = map(int, input().split())
    edge.append((w, u, v))
edge.sort() # 가중치를 기준으로 오름차순 정렬
p = [i for i in range(V+1)] # 대표원소 초기화
# N개의 정점이 있으면 사이클이 생기지 않도록 N-1 개의 간선을 선택
# mst에 포함된 간선의 가중치의 합 구하기
cnt = 0
total = 0 # 가중치의 합
for w, u, v in edge: # N-1개의 간선 선택 루프
    if find_set(u) != find_set(v): # 사이클을 형성하지 않으면 선택(서로 다른 집합에 속해 있으면)
        cnt += 1
        total += w # 가중치의 합
        p[find_set(v)] = find_set(u) # v의 대표원소를 u의 대표원소로 바꿈(union)
        if cnt == V:
            break
print(total)

6 11
0 1 32
0 2 31
0 5 60
0 6 51
1 2 21
2 4 46
2 6 25
3 4 34
3 5 18
4 5 40
4 6 51
175


###  해설 2